# обнаружение инсайдерских угроз, которая использует комбинацию GCN (Graph Convolutional Network), LSTM, и GRU.

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

# GCN + LSTM + GRU модель

In [8]:
class InsiderThreatModel(nn.Module):
    def __init__(self, in_feats, hidden_gcn, hidden_rnn, num_classes):
        super(InsiderThreatModel, self).__init__()
        self.gcn1 = GCNConv(in_feats, hidden_gcn)
        self.gcn2 = GCNConv(hidden_gcn, hidden_gcn)

        self.lstm = nn.LSTM(hidden_gcn, hidden_rnn, batch_first=True)
        self.gru = nn.GRU(hidden_rnn, hidden_rnn, batch_first=True)

        self.fc = nn.Linear(hidden_rnn, num_classes)

    def forward(self, x, edge_index, seq_lengths):
        # GCN layers
        x = F.relu(self.gcn1(x, edge_index))
        x = F.relu(self.gcn2(x, edge_index))

        # Упаковываем последовательности
        x = x.unsqueeze(1)  # [N, 1, F]
        lstm_out, _ = self.lstm(x)
        gru_out, _ = self.gru(lstm_out)

        out = self.fc(gru_out[:, -1, :])  # берем последний временной шаг
        return F.log_softmax(out, dim=1)

# Пример генерации графа (данные нужно адаптировать под реальные события)

In [9]:
def create_example_data():
    # 5 узлов, 4 признака
    x = torch.rand((5, 4), dtype=torch.float)
    edge_index = torch.tensor([[0, 1, 2, 3, 4, 1], [1, 0, 3, 4, 0, 3]], dtype=torch.long)

    # Метки: 0 - нормальный, 1 - угроза
    y = torch.tensor([0, 0, 1, 0, 1], dtype=torch.long)
    data = Data(x=x, edge_index=edge_index, y=y)
    return data

# Обучение модели

In [10]:
def train_model():
    data = create_example_data()
    model = InsiderThreatModel(in_feats=4, hidden_gcn=16, hidden_rnn=8, num_classes=2)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.NLLLoss()

    model.train()
    for epoch in range(100):
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, None)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

    print("Training complete.")

In [11]:
train_model()

Epoch 0, Loss: 0.7541
Epoch 10, Loss: 0.6770
Epoch 20, Loss: 0.6673
Epoch 30, Loss: 0.6280
Epoch 40, Loss: 0.5388
Epoch 50, Loss: 0.3837
Epoch 60, Loss: 0.2290
Epoch 70, Loss: 0.1105
Epoch 80, Loss: 0.0402
Epoch 90, Loss: 0.0167
Training complete.
